In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2

In [2]:
def create_base_network(input_shape):
    input = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    return models.Model(input, x)

In [3]:
input_shape = (105, 105, 1)
base_network = create_base_network(input_shape)

In [4]:
input_a = tf.keras.Input(shape=input_shape)
input_b = tf.keras.Input(shape=input_shape)

In [5]:
processed_a = base_network(input_a)
processed_b = base_network(input_b)

In [6]:
distance = layers.Lambda(lambda embeddings: tf.keras.backend.abs(embeddings[0] - embeddings[1]))([processed_a, processed_b])
outputs = layers.Dense(1, activation='sigmoid')(distance)
model = models.Model([input_a, input_b], outputs)


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
def load_image(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (105, 105))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

In [9]:
# Prepare your pairs of images and labels for training
def create_pairs(base_image_path, other_image_paths, same_class_paths):
    base_image = load_image(base_image_path)
    pairs = []
    labels = []

    for other_image_path in other_image_paths:
        other_image = load_image(other_image_path)
        pairs.append([base_image, other_image])
        labels.append(0)  # Different class

    for same_class_path in same_class_paths:
        same_class_image = load_image(same_class_path)
        pairs.append([base_image, same_class_image])
        labels.append(1)  # Same class

    return np.array(pairs), np.array(labels)


In [11]:
base_image_path = r"C:\Users\HP\OneDrive\Desktop\proj1\katrina2.png"
other_image_paths = [r"C:\Users\HP\OneDrive\Desktop\proj1\tripti1.png", r"C:\Users\HP\OneDrive\Desktop\proj1\tripti2.png"]
same_class_paths = [r"C:\Users\HP\OneDrive\Desktop\proj1\katrina1.png", r"C:\Users\HP\OneDrive\Desktop\proj1\katrina2.png"]

pairs, labels = create_pairs(base_image_path, other_image_paths, same_class_paths)
pairs = [pairs[:, 0], pairs[:, 1]]


In [12]:
pairs, labels = create_pairs(base_image_path, other_image_paths, same_class_paths)
pairs = [pairs[:, 0], pairs[:, 1]]

In [13]:
model.fit(pairs, labels, epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6921 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 160ms/step - loss: 0.5784 - accuracy: 0.7500
Epoch 3/10
1/1 [==============================] - 0s 156ms/step - loss: 0.3771 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 154ms/step - loss: 0.2222 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 157ms/step - loss: 0.1787 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 157ms/step - loss: 0.1733 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 150ms/step - loss: 0.1727 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 149ms/step - loss: 0.1725 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 149ms/step - loss: 0.1724 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 154ms/step - loss: 0.1723 - accuracy: 1.0000


In [15]:
img1 = load_image(r"C:\Users\HP\OneDrive\Desktop\proj1\katrina2.png")
img2 = load_image(r"C:\Users\HP\OneDrive\Desktop\proj1\tripti2.png")
prediction = model.predict([np.expand_dims(img1, axis=0), np.expand_dims(img2, axis=0)])
print(f'Prediction: {prediction}')

1/1 [==============================] - 0s 165ms/step
Prediction: [[5.755671e-10]]


In [16]:
val_other_image_paths = [r"C:\Users\HP\OneDrive\Desktop\proj1\tripti1.png", r"C:\Users\HP\OneDrive\Desktop\proj1\tripti2.png"]
val_same_class_paths = [r"C:\Users\HP\OneDrive\Desktop\proj1\katrina1.png", r"C:\Users\HP\OneDrive\Desktop\proj1\katrina2.png"]

val_pairs, val_labels = create_pairs(base_image_path, val_other_image_paths, val_same_class_paths)
val_pairs = [val_pairs[:, 0], val_pairs[:, 1]]

# Evaluate the model
loss, accuracy = model.evaluate(val_pairs, val_labels)

print(f'Validation Accuracy: {accuracy}')

1/1 [==============================] - 0s 264ms/step - loss: 0.1721 - accuracy: 1.0000
Validation Accuracy: 1.0
